# <center>Site-level Study on EU ETS Policy Effects:</center>
## <center>Dual-Outcome Analysis (ETS CO₂ + Satellite NOx)</center>

## Research question / Hypotheses

**Core question:**
> How do EU ETS carbon market stringency interventions affect both verified CO₂ emissions and satellite-derived NOx emission proxies around major industrial emitters?

**Dual Outcomes:**
1. **ETS CO₂** (ground-truth): Verified annual emissions from EU ETS registry
2. **Satellite NOx** (proxy): Beirle-style flux-divergence NOx estimates from TROPOMI

**Sub-questions / hypotheses:**
- *H1:* Allocation shortfall (ratio < 1) induces facilities to reduce combustion, lowering both CO₂ and NOx.
- *H2:* The magnitude of emission reduction correlates with plant characteristics (fuel type, capacity) and local geography.
- *H3:* Treatment effects may differ between ground-truth CO₂ and noisy satellite NOx due to measurement error attenuation.

## Variables and Causal Structure

We work with a **plant–year panel** indexed by facility *i* and year *t*.

### Dual Outcomes
- **$Y^{CO2}_{it}$**: Verified annual CO₂ emissions from EU ETS registry (tCO₂/yr, log-transformed).
  Ground-truth administrative data with high reliability.
- **$Y^{NOx}_{it}$**: Satellite-derived NOx emission proxy (kg/s) via Beirle-style flux-divergence.
  Noisy but independent measure of combustion intensity.

### Treatment (Causal Target)
- **Policy $P_{it}$**: EU ETS stringency at the plant–year level, measured by the
  allocation ratio (free allocation / verified emissions). Values < 1 indicate the
  facility must purchase additional allowances, creating abatement incentives.

### Observed Covariates
- **Plant characteristics $X_{it}$**: Time-varying capacity (MW) and fuel mix shares.
- **Geographic context $G_i$**: High-dimensional AlphaEarth embeddings (64-dim) encoding
  land use, infrastructure, and climate patterns. Static controls for between-unit heterogeneity.

### Unobserved Variables
- **$U_i$**: Plant-level time-invariant unobservables (baseline technology, combustion efficiency).
- **$U_{it}$**: **Plant-level time-varying unobservables**—the key identification challenge.
  Includes dispatch/utilization, maintenance status, operational efficiency changes.
- **$U_{rt}$**: Region–time effects (electricity demand, fuel prices, regional policy enforcement).

### Why Dual Outcomes?

1. **ETS CO₂** is ground-truth but may be subject to reporting incentives
2. **Satellite NOx** is independent but noisy (~35-45% uncertainty)
3. Consistent effects across both outcomes strengthen causal claims
4. Divergent effects may reveal measurement error attenuation in NOx

```mermaid
flowchart LR
  %% Unobserved
  subgraph Unobserved
    direction TB
    Ui["U_i: time-invariant (technology, efficiency)"]
    Uit["U_it: time-varying (dispatch, maintenance, efficiency)"]
    Urt["U_rt: region–time (demand, prices, politics)"]
  end

  %% Observed
  subgraph Observed Controls
    direction TB
    X["X_it: capacity, fuel mix"]
    G["G_i: AlphaEarth embeddings"]
    W["W_it: wind (measurement)"]
  end

  %% Treatment and Outcome
  subgraph Treatment
    P["P_it: Allocation ratio"]
  end

  subgraph Outcome
    Y["Y_it: NO₂ enhancement"]
  end

  %% === CAUSAL ARROWS ===

  %% Time-invariant unobserved
  Ui --> X
  Ui --> Y
  Ui -.->|"absorbed by facility FE"| P

  %% Plant-level time-varying unobserved (THE KEY CHALLENGE)
  Uit --> Y
  Uit --> P
  Urt --> Uit

  %% Region-time effects
  Urt --> P
  Urt --> Y
  Urt -.->|"absorbed by region×year FE"| X

  %% Observed controls
  X --> P
  X --> Y
  G --> Y
  W --> Y

  %% CAUSAL EFFECT OF INTEREST
  P ==>|"β (causal target)"| Y

  %% Mediation - policy affects dispatch via carbon costs in bids
  P -.->|"mediation via dispatch"| Uit
```

---

## Identification Strategy

### What We Control For

| Variable | Absorbed By | Rationale |
|----------|-------------|-----------|
| $U_i$ (time-invariant) | Facility FE | Technology, location, baseline efficiency |
| $U_{rt}$ (region-time) | Region×Year FE | Demand shocks, fuel prices, regional policy |
| $X_{it}$ (observed) | Controls | Capacity, fuel mix |
| $G_i$ (geography) | AlphaEarth embeddings | Land use, infrastructure, climate context |

### What We Do NOT Control For: $U_{it}$

Plant-level time-varying unobservables ($U_{it}$) include dispatch, maintenance, and efficiency changes.
We deliberately do not control for these because:

1. **Dispatch as confounder**: Regional demand → higher dispatch → more emissions → lower $R_{it}$.
   Same demand → more combustion → higher NO₂. This creates spurious correlation.

2. **Dispatch as mediator**: Policy affects dispatch via carbon costs in bids (higher costs → higher bids →
   lower dispatch probability). Thus $P_{it} \to U_{it} \to Y_{it}$. Controlling for $U_{it}$ blocks
   this pathway and biases $\hat{\beta}$ toward zero.

3. **Facility×Year FE infeasible**: Would absorb all within-facility-year variation, including treatment.

4. **Interactive FE risky**: Estimating facility-specific loadings on common factors may open
   backdoor paths if loadings correlate with allocation mechanism.

### Our Solution: Region×Year FE

Region×Year FE absorbs the *common regional component* of $U_{it}$ (since dispatch responds to
regional demand/prices) without estimating facility-specific parameters. The identifying variation:

> *Within the same region and year, do facilities with different allocation ratios show different NO₂?*

This leaves **facility-specific deviations** in $U_{it}$ as residual confounding (e.g., idiosyncratic
outages, plant-specific demand). These are plausibly second-order and orthogonal to allocation ratio
conditional on capacity and fuel mix controls.

---

## Analysis Methodology

### Treatment Definition

**Continuous Treatment**: `eu_alloc_ratio = allocated_allowances / verified_emissions`
- Ratio < 1 → Facility must purchase additional allowances (treated)
- Ratio ≥ 1 → Facility has sufficient free allocation (control/less treated)

**Discrete Treatment** (for Callaway-Sant'Anna):
- `treated = 1` if `eu_alloc_ratio < 1` in a given year
- `cohort` = first year when `treated = 1` (staggered adoption)

### Clustering Strategy

We use **PyPSA-Eur power system regions** which are created via k-means clustering 
on the European transmission network. These regions are used for:
1. **Clustered standard errors** — accounts for within-region correlation
2. **Region × Year fixed effects** — absorbs region-specific time shocks (Spec 2)

### Three Core Specifications

| # | Specification | Treatment | Fixed Effects | Std Errors |
|---|---|---|---|---|
| **1** | TWFE (basic) | `eu_alloc_ratio` | Facility + Year | Clustered by PyPSA region |
| **2** | TWFE (strict) | `eu_alloc_ratio` | Facility + Region×Year | Clustered by PyPSA region |
| **3** | Callaway-Sant'Anna | Binary (ratio < 1) | Built-in | Clustered by PyPSA region |

**Note on SE clustering with Region FE (Spec 2)**: We cluster SEs by the same 
PyPSA regions used for FE. This accounts for within-region serial correlation.

## 1. Setup and Data Loading

In [ ]:
# =============================================================================
# Imports
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import sys
sys.path.insert(0, '.')

# Config: column names and paths
from src.analysis.config import (
    FAC_ID_COL, YEAR_COL, CLUSTER_COL, ALLOC_RATIO_COL,
    ETS_CO2_COL, LOG_ETS_CO2_COL,
    NOX_OUTCOME_COL, NOX_SE_COL, DL_CONSERVATIVE_COL
)

# =============================================================================
# ANALYSIS PARAMETERS
# =============================================================================

# Time range
START_YEAR = 2018
END_YEAR = 2023

# Treatment
TREATMENT_COL = ALLOC_RATIO_COL        # "eu_alloc_ratio"
TREATMENT_THRESHOLD = 1.0               # Treated if ratio < 1

# Dual Outcomes
OUTCOMES = {
    "ETS_CO2": LOG_ETS_CO2_COL,     # Log verified emissions (ground truth)
    "Satellite_NOx": NOX_OUTCOME_COL,  # Beirle-style NOx (kg/s)
}

# Controls
CONTROLS = ["capacity_mw", "share_coal", "share_gas"]

# Clustering (PyPSA-Eur k-means regions)
PYPSA_RESOLUTION = "128"  # 37, 64, 128, or 256 regions

# Sample restrictions
MIN_YEARS_PER_FACILITY = 3

# Satellite-specific filters
SATELLITE_DETECTION_LIMIT = "conservative"  # "conservative" (0.11 kg/s) or "permissive" (0.03 kg/s)

print("Parameters configured:")
print(f"  Years: {START_YEAR}-{END_YEAR}")
print(f"  Treatment: {TREATMENT_COL} < {TREATMENT_THRESHOLD}")
print(f"  Outcomes: {list(OUTCOMES.keys())}")
print(f"  PyPSA resolution: {PYPSA_RESOLUTION} regions")
print(f"  Satellite DL: {SATELLITE_DETECTION_LIMIT}")

In [ ]:
# =============================================================================
# Load Data (ETS + Satellite outcomes merged)
# =============================================================================
from src.analysis.data import load_analysis_panel, load_facilities_static

# Load panel with satellite outcomes included
panel = load_analysis_panel(include_satellite=True)
static = load_facilities_static()

print(f"\\nPanel: {len(panel)} obs, {panel[FAC_ID_COL].nunique()} facilities")
print(f"Years: {panel[YEAR_COL].min()} - {panel[YEAR_COL].max()}")
print(f"\\nAvailable outcomes:")
print(f"  ETS CO₂: {ETS_CO2_COL} → {panel[ETS_CO2_COL].notna().sum()} valid obs")
if NOX_OUTCOME_COL in panel.columns:
    print(f"  Satellite NOx: {NOX_OUTCOME_COL} → {panel[NOX_OUTCOME_COL].notna().sum()} valid obs")

## 2. Data Preparation and Treatment Construction

In [ ]:
# =============================================================================
# Build Treatment Variables
# =============================================================================
from src.analysis.data import build_treatment_variables, apply_sample_filters

# Continuous: eu_alloc_ratio (already in data)
# Discrete: treated = 1 if ratio < 1, cohort = first year treated
panel = build_treatment_variables(
    panel, 
    treatment_col=TREATMENT_COL,
    threshold=TREATMENT_THRESHOLD
)

# Apply sample filters (ETS outcome required for base sample)
panel = apply_sample_filters(
    panel, 
    min_years=MIN_YEARS_PER_FACILITY,
    year_range=(START_YEAR, END_YEAR),
    require_outcome=True,
    outcome_col=ETS_CO2_COL  # Filter on ETS CO₂ (ground truth)
)

In [ ]:
# =============================================================================
# Panel Summary
# =============================================================================
print(f"\nFinal panel:")
print(f"  Observations: {len(panel)}")
print(f"  Facilities: {panel[FAC_ID_COL].nunique()}")
print(f"  Years: {panel[YEAR_COL].min()}-{panel[YEAR_COL].max()}")
print(f"  Treated (ever): {(panel.groupby(FAC_ID_COL)['treated'].max() > 0).sum()}")
print(f"  Never treated: {(panel.groupby(FAC_ID_COL)['treated'].max() == 0).sum()}")

## 3. Assign PyPSA-Eur Regions

PyPSA-Eur uses **k-means clustering** on the European transmission network to create
power system regions. These are our clustering units for:
- **Clustered standard errors** (within-region correlation)  
- **Region × Year fixed effects** (Specification 2)

In [ ]:
# =============================================================================
# Assign PyPSA-Eur Regions (k-means clustered power system)
# =============================================================================
from src.analysis.clusters import assign_pypsa_clusters, get_cluster_summary

try:
    panel = assign_pypsa_clusters(
        panel, static, 
        resolution=PYPSA_RESOLUTION,
        cluster_col=CLUSTER_COL
    )
    print(f"\nCluster summary:")
    display(get_cluster_summary(panel, cluster_col=CLUSTER_COL))
except FileNotFoundError as e:
    print(f"PyPSA clusters not found: {e}")
    print("Falling back to country_code as cluster variable")
    panel[CLUSTER_COL] = static.set_index(FAC_ID_COL)["country_code"].reindex(panel[FAC_ID_COL]).values

## 4. TWFE Continuous Specifications

Two TWFE specifications for each outcome:
1. **Spec 1**: Facility + Year FE, clustered SEs by PyPSA region
2. **Spec 2**: Facility + Region×Year FE, clustered SEs by PyPSA region

In [ ]:
# =============================================================================
# Run Dual-Outcome TWFE Analysis
# =============================================================================
from src.analysis.continuous import run_dual_outcome_analysis, format_dual_results_table
from src.analysis.data import apply_satellite_filters, get_intersection_sample

# Run TWFE on FULL ETS sample (ground truth)
print("\\n" + "=" * 70)
print("FULL ETS SAMPLE (ground truth CO₂)")
print("=" * 70)
dual_results_full = run_dual_outcome_analysis(
    panel,
    treatment_col=TREATMENT_COL,
    controls=CONTROLS,
    cluster_col=CLUSTER_COL,
    ets_col=LOG_ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL
)

# Display results table
print("\\n" + "=" * 60)
print("TWFE Results Summary (Full Sample)")
print("=" * 60)
display(format_dual_results_table(dual_results_full))

In [ ]:
# =============================================================================
# Run TWFE on INTERSECTION Sample (both outcomes valid, above detection limit)
# =============================================================================

# Get intersection sample with satellite filters
intersection_panel = get_intersection_sample(
    panel,
    ets_col=ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL,
    detection_limit=SATELLITE_DETECTION_LIMIT
)

if len(intersection_panel) > 50:
    print("\\n" + "=" * 70)
    print("INTERSECTION SAMPLE (both ETS CO₂ and Satellite NOx valid)")
    print("=" * 70)
    
    dual_results_intersection = run_dual_outcome_analysis(
        intersection_panel,
        treatment_col=TREATMENT_COL,
        controls=CONTROLS,
        cluster_col=CLUSTER_COL,
        ets_col=LOG_ETS_CO2_COL,
        nox_col=NOX_OUTCOME_COL
    )
    
    print("\\n" + "=" * 60)
    print("TWFE Results Summary (Intersection Sample)")
    print("=" * 60)
    display(format_dual_results_table(dual_results_intersection))
else:
    print(f"Intersection sample too small ({len(intersection_panel)} obs)")

## 5. Discrete Treatment: Callaway-Sant'Anna DiD

Estimate treatment effects using the Callaway & Sant'Anna (2021) estimator for 
staggered difference-in-differences with heterogeneous treatment timing.

**Treatment event**: First year when `eu_alloc_ratio < 1`

Reference: [Callaway & Sant'Anna (2021)](https://psantanna.com/files/Callaway_SantAnna_2020.pdf)

In [ ]:
# =============================================================================
# Validate Cohorts for Callaway-Sant'Anna
# =============================================================================
from src.analysis.csdid import build_cohorts, validate_cohorts

# Ensure cohorts are properly constructed (already done in build_treatment_variables)
cohort_validation = validate_cohorts(panel, cohort_col="cohort")

In [ ]:
# =============================================================================
# Callaway-Sant'Anna Estimation (Both Outcomes, Both SE Specs)
# =============================================================================
from src.analysis.csdid import run_csdid_both_specs

csdid_results = {}

for outcome_name, outcome_col in OUTCOMES.items():
    print(f"\n{'#' * 60}")
    print(f"# CS-DiD: {outcome_name}")
    print(f"{'#' * 60}")
    
    try:
        csdid_results[outcome_name] = run_csdid_both_specs(
            panel,
            outcome_col=outcome_col,
            cohort_col="cohort",
            cluster_col=CLUSTER_COL,
            control_group="nevertreated",
            est_method="dr"
        )
    except ImportError as e:
        print(f"Error: {e}")
        print("Install with: pip install csdid")
        csdid_results[outcome_name] = None

In [ ]:
# =============================================================================
# Event Study Plots
# =============================================================================
from src.analysis.csdid import plot_event_study, test_pre_trends

for outcome_name, results in csdid_results.items():
    if results is None:
        continue
    
    # Use the no_cluster result for event study
    res = results.get("no_cluster")
    if res is None:
        continue
    
    print(f"\n{outcome_name} Event Study:")
    fig = plot_event_study(
        res,
        title=f"Event Study: ETS → {outcome_name}",
        max_pre=3,
        max_post=4
    )
    plt.show()
    
    # Pre-trends test
    test_pre_trends(res, n_pre_periods=2)

## 6. Covariate Balance and Diagnostics

In [ ]:
# =============================================================================
# Covariate Summary by Treatment Status
# =============================================================================
# Simple balance check
covariates = ["capacity_mw", "share_coal", "share_gas", "share_oil", "share_biomass"]
available_covs = [c for c in covariates if c in panel.columns]

print("Mean by treatment status (baseline year):")
baseline = panel.groupby(FAC_ID_COL).first()
print(baseline.groupby("treated")[available_covs].mean().T)

In [ ]:
# =============================================================================
# Outcome Trends: Treated vs Never Treated
# =============================================================================
for outcome_name, outcome_col in OUTCOMES.items():
    if outcome_col not in panel.columns:
        continue
    
    # Mean outcome by year and treatment status
    trends = panel.groupby([YEAR_COL, "treated"])[outcome_col].mean().unstack()
    
    fig, ax = plt.subplots(figsize=(10, 5))
    trends.plot(ax=ax, marker='o')
    ax.set_xlabel("Year")
    ax.set_ylabel(outcome_name)
    ax.set_title(f"{outcome_name} Trends: Treated vs Never-Treated")
    ax.legend(["Never Treated", "Ever Treated"])
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6b. Outcome Correlation Diagnostic

Check correlation between ETS CO₂ (ground truth) and Satellite NOx (proxy).
- Positive correlation suggests both capture combustion activity
- Correlation strength indicates signal-to-noise in satellite proxy

In [ ]:
# =============================================================================
# ETS CO₂ vs Satellite NOx Correlation
# =============================================================================
from scipy import stats

# Get intersection sample (both outcomes valid)
corr_df = panel.dropna(subset=[LOG_ETS_CO2_COL, NOX_OUTCOME_COL])

if len(corr_df) > 10:
    # Compute correlations
    pearson_r, pearson_p = stats.pearsonr(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL])
    spearman_r, spearman_p = stats.spearmanr(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL])
    
    print(f"Outcome Correlation (N = {len(corr_df)} obs):")
    print(f"  Pearson r  = {pearson_r:.3f} (p = {pearson_p:.4f})")
    print(f"  Spearman ρ = {spearman_r:.3f} (p = {spearman_p:.4f})")
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL], 
               alpha=0.4, s=20, c='steelblue')
    
    # Fit line
    slope, intercept, _, _, _ = stats.linregress(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL])
    x_line = np.linspace(corr_df[LOG_ETS_CO2_COL].min(), corr_df[LOG_ETS_CO2_COL].max(), 100)
    ax.plot(x_line, slope * x_line + intercept, 'r-', lw=2, label=f'OLS: β={slope:.4f}')
    
    ax.set_xlabel('Log ETS CO₂ (tCO₂/yr)')
    ax.set_ylabel('Satellite NOx (kg/s)')
    ax.set_title(f'ETS CO₂ vs Satellite NOx Correlation\\n(Pearson r = {pearson_r:.3f})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # By fuel type (if available)
    if 'share_coal' in panel.columns:
        coal_mask = corr_df['share_coal'] > 0.5
        if coal_mask.sum() > 10 and (~coal_mask).sum() > 10:
            r_coal, _ = stats.pearsonr(corr_df.loc[coal_mask, LOG_ETS_CO2_COL], 
                                        corr_df.loc[coal_mask, NOX_OUTCOME_COL])
            r_other, _ = stats.pearsonr(corr_df.loc[~coal_mask, LOG_ETS_CO2_COL], 
                                         corr_df.loc[~coal_mask, NOX_OUTCOME_COL])
            print(f"\\nCorrelation by fuel type:")
            print(f"  Coal-dominated (>50%): r = {r_coal:.3f} (N = {coal_mask.sum()})")
            print(f"  Other fuels:           r = {r_other:.3f} (N = {(~coal_mask).sum()})")
else:
    print("Insufficient data for correlation analysis")

## 7. Robustness Checks

### Methodological Notes

**Clustering SEs vs Region FE:**
- **Clustered SEs** (by PyPSA region or k-means cluster): Accounts for within-cluster correlation in errors. Appropriate when units within clusters are correlated but treatment varies within clusters.
- **Region FE** (country × year): Absorbs region-specific time-varying shocks. Use when regional factors (electricity prices, policy enforcement) affect both treatment and outcome.

**Both approaches are valid but address different concerns:**
- Clustered SEs: Inference problem (correct standard errors)
- Region FE: Identification problem (absorb confounders)

We run both separately.

In [ ]:
# =============================================================================
# Summary: Dual-Outcome Results
# =============================================================================
print("\\n" + "=" * 70)
print("FINAL RESULTS SUMMARY: DUAL OUTCOMES")
print("=" * 70)

# TWFE Results
print("\\n### TWFE Continuous Treatment ###")
if 'dual_results_full' in dir():
    display(format_dual_results_table(dual_results_full))

# CS-DiD Results (if run)
print("\\n### Callaway-Sant'Anna Discrete Treatment ###")
if 'csdid_results' in dir() and csdid_results:
    for outcome_name, results in csdid_results.items():
        if results is None:
            print(f"{outcome_name}: Not estimated (csdid package required)")
            continue
        
        for spec_name, res in results.items():
            att = res["agg_simple"]["att"]
            se = res["agg_simple"]["se"]
            print(f"{outcome_name} ({spec_name}): ATT = {att:.6f} (SE = {se:.6f})")

## 8. Summary and Next Steps

### Specifications Run

**TWFE Continuous (per outcome):**
1. Facility + Year FE, region-clustered SEs
2. Facility + Region×Year FE, region-clustered SEs

**Callaway-Sant'Anna Discrete (per outcome):**
1. Unit-level SEs
2. Region-clustered SEs

**Dual Outcomes:**
- **ETS CO₂**: `log_ets_co2` — log verified emissions (ground truth)
- **Satellite NOx**: `beirle_nox_kg_s` — Beirle-style flux-divergence proxy

### Interpretation Notes

1. **ETS CO₂ effects** are more reliable (ground-truth administrative data)
2. **Satellite NOx effects** may be attenuated due to ~35-45% measurement error
3. **Consistent signs** across outcomes strengthen causal claims
4. **Detection limit filter** (0.11 kg/s conservative) may introduce selection

### Next Steps
1. **Run Data Pipeline.ipynb** to generate satellite outcome data
2. **Install packages**: `pip install pyfixest csdid geopandas`
3. **Heterogeneity analysis** by fuel type, country, capacity
4. **Sensitivity analysis** with different treatment thresholds
5. **Compare ETS vs NOx** on intersection sample